In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2
import base64
import time
import math
import datetime
import os
from PIL import Image
from io import BytesIO
from tqdm import tqdm
from glob import glob

import torch
import torchvision
import torch.distributed as dist
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from collections import defaultdict, deque

In [2]:
!unzip '/content/drive/MyDrive/machinelearning/병변검출_yolo/병변 검출_data.zip'

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/train_157622.json  
  inflating: train/train_157623.json  
  inflating: train/train_157624.json  
  inflating: train/train_157625.json  
  inflating: train/train_157626.json  
  inflating: train/train_157627.json  
  inflating: train/train_157628.json  
  inflating: train/train_157629.json  
  inflating: train/train_157630.json  
  inflating: train/train_157631.json  
  inflating: train/train_157632.json  
  inflating: train/train_157633.json  
  inflating: train/train_157634.json  
  inflating: train/train_157635.json  
  inflating: train/train_157636.json  
  inflating: train/train_157637.json  
  inflating: train/train_157638.json  
  inflating: train/train_157639.json  
  inflating: train/train_157640.json  
  inflating: train/train_157641.json  
  inflating: train/train_157642.json  
  inflating: train/train_157643.json  
  inflating: train/train_157644.json  
  inflating: train/train_157645.json  
  inflating: train/train_157

In [3]:
train_files = glob('./train/*')
test_files = glob('./test/*')

In [4]:
train_json_list = []
for file in tqdm(train_files):
  with open(file,"r") as f:
    train_json_list.append(json.load(f))

test_json_list = []
for file in tqdm(test_files):
  with open(file,"r") as f:
    test_json_list.append(json.load(f))

100%|██████████| 20874/20874 [00:03<00:00, 5431.51it/s]


In [5]:
label_count = {}
for data in train_json_list:
    for shape in data['shapes']:
        try:
            label_count[shape['label']]+=1
        except:
            label_count[shape['label']]=1

In [6]:
label_count

{'01_ulcer': 38958, '05_bleeding': 19498, '02_mass': 9760, '04_lymph': 12002}

In [7]:
train_json_list[0]

{'file_name': 'train_148746.json',
 'shapes': [{'label': '01_ulcer',
   'points': [[258.6568047337278, 424.2603550295858],
    [394.0298507462686, 424.2603550295858],
    [394.0298507462686, 542.0118343195267],
    [258.6568047337278, 542.0118343195267]]}],
 'imageData': '...',
 'imageHeight': 576,
 'imageWidth': 576}

In [8]:
# 이미지 정보는 base64로 json파일에 포함
plt.figure(figsize=(25,30))
for i in range(30):  # 6x5
  plt.subplot(6,5,i+1)
  # base64를 array형태로 변환
  img =  Image.open(BytesIO(base64.b64decode(train_json_list[i]['imageData'])))
  img = np.array(img,np.uint8)
  title = []
  for shape in train_json_list[i]['shapes']:
    points =  np.array(shape['points'],np.int32)
    cv2.polylines(img,[points],True,(0,255,0),3)
    title.append(shape['label'])
  title = ','.join(title)
  plt.imshow(img)
  plt.subplot(6,5,i+1).set_title(title)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [9]:
# 데이터 셋
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, json_list,mode = 'train'):
        self.mode = mode
        self.file_name = [json_file['file_name'] for json_file in json_list]


    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = read_image(img_path)
        mask = read_image(mask_path)
        # instances are encoded as different colors
        obj_ids = torch.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        num_objs = len(obj_ids)

        # split the color-encoded mask into a set
        # of binary masks
        masks = (mask == obj_ids[:, None, None]).to(dtype=torch.uint8)

        # get bounding box coordinates for each mask
        boxes = masks_to_boxes(masks)

        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = idx
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Wrap sample and targets into torchvision tv_tensors:
        img = tv_tensors.Image(img)

        target = {}
        target["boxes"] = tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=F.get_size(img))
        target["masks"] = tv_tensors.Mask(masks)
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)


ImportError: ignored